In [81]:
# adding pyspark to sys.path at runtime
import findspark
findspark.init('')

In [80]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
                    .appName('resp-entregavel_2')\
                    .getOrCreate()

### Abaixo temos as informações fornecidas através da imagem sobre o conjunto de dados que iremos trabalhar.

![](src/data.png)

In [88]:
# Transcrevendo os dados
transacoes = [{'transacao_id':1, 'total_bruto':3000, 'desconto_percentual':6.99},
              {'transacao_id':2, 'total_bruto':57989, 'desconto_percentual':1.45},
              {'transacao_id':4, 'total_bruto':1, 'desconto_percentual':None},
              {'transacao_id':5, 'total_bruto':34, 'desconto_percentual':0.0}]

Agora que já temos os dados em formato de 'lista de dicionários', vamos criar um dataframe usando a função do Spark createDataframe e já aproveitando preenchendo o valor nulo com 0, através da função na.fill, e logo em seguida vamos mostrar os dados.

In [95]:
df = spark.createDataFrame(transacoes).na.fill(0)
df.show()

+-------------------+-----------+------------+
|desconto_percentual|total_bruto|transacao_id|
+-------------------+-----------+------------+
|               6.99|       3000|           1|
|               1.45|      57989|           2|
|                0.0|          1|           4|
|                0.0|         34|           5|
+-------------------+-----------+------------+



Para entender melhor sobre a estrutura do dataframe podemos usar a função printSchema.

In [93]:
df.printSchema()

root
 |-- desconto_percentual: double (nullable = true)
 |-- total_bruto: long (nullable = true)
 |-- transacao_id: long (nullable = true)



Agora precisamos calcular o total líquido da empresa. Esse total é calculado da seguinte forma total_liquido = soma(total_bruto – desconto_percentual).

O resultado esperado é um total liquido de 59973.46.

Portanto, vamos seguir a mesma idea da atividade anterior, vamos calcular o desconto (100-desconto_percentual)% e multiplicar o resultado pelo total bruto.

In [118]:
df = df.withColumn('total_liquido', ( df['total_bruto'] * (100 - df['desconto_percentual'])/100))

df.show()



+-------------------+-----------+------------+-------------+
|desconto_percentual|total_bruto|transacao_id|total_liquido|
+-------------------+-----------+------------+-------------+
|               6.99|       3000|           1|       2790.3|
|               1.45|      57989|           2|   57148.1595|
|                0.0|          1|           4|          1.0|
|                0.0|         34|           5|         34.0|
+-------------------+-----------+------------+-------------+



Agora para calcular a soma do total_liquido, precisamos importar a função sum de pyspark.sql.functions e fazer o cálculo.

In [115]:
import pyspark.sql.functions as f

df.select(f.sum('total_liquido')).show()

+------------------+
|sum(total_liquido)|
+------------------+
|59973.459500000004|
+------------------+



E assim obtemos a resposta, que é 59973.459500000004 :)